## 1-数据预处理 | 为训练数据添加 `<|endoftext|>`
- 将训练集中每个故事添加结束标志，防止故事间跨越，学习全局超长语料
- 依然是全局BPE，但是故事间有 `<|endoftext|>` 作为挡板
- **不是** 每个故事单独训练一个BPE词表再汇总\[这可能会导致出现不同的token编号，模型无法同一使用\]，而是全局统计

In [3]:
train_raw_path = "/home/winbeau/Study/1-transformer/datasets/TinyStories/txt/train.txt"
valid_raw_path = "/home/winbeau/Study/1-transformer/datasets/TinyStories/txt/valid.txt"

# 查看前几行
def preview_txt(cnt_line, txt_path=train_raw_path): 
    with open(txt_path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f): # 行号 和 内容
            if i >= cnt_line:
                break
            print(line.strip())

In [4]:
preview_txt(2)

One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.  Lily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."  Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.
Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.  One day, Beep was driving in the park when he saw a big tree. The tree had many leaves that were falling. B

In [5]:
import os 

base_dir = "/home/winbeau/Study/1-transformer/datasets/TinyStories/txt/"

files = [
    ("train.txt", "train_with_eot.txt"), 
    ("valid.txt", "valid_with_eot.txt"),
]

In [6]:
def add_endoftext(infile: str, outfile: str): 
    """行末添加 <|endoftext|> , 忽略空行 """
    cnt_in, cnt_out = 0, 0
    with open(infile, "r", encoding="utf-8") as fin, open(outfile, "w", encoding="utf=8") as fout: 
        for line in fin: 
            text = line.strip()
            cnt_in += 1
            if text: 
                fout.write(text + "<|endoftext|>\n")
                cnt_out += 1
    print(f"Complete! {cnt_out} / {cnt_in}")

In [28]:
for fin, fout in files: 
    add_endoftext(
        os.path.join(base_dir, fin),
        os.path.join(base_dir, fout) 
    )
print("All files have added <|endoftext|> and saved!")

Complete! 2119489 / 2119719
Complete! 21990 / 21990
All files have added <|endoftext|> and saved!


In [30]:
preview_txt(2, os.path.join(base_dir, "train_with_eot.txt")) # 验证是否加入成功

One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.  Lily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."  Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.<|endoftext|>
Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.  One day, Beep was driving in the park when he saw a big tree. The tree had many leaves that we

## 2-BPE | 预分词(Pre-tokenization) \[ 多进程并行 \]
<input type="checkbox" unchecked> BPE-No.1: 预分词(Pre-tokenization) 把原始文本分成初步"词形片段"并计数(正则化去掉标点、符号)

<input type="checkbox" unchecked> BPE-No.2: 统计字节对频率(Pair counting) 在"词形片段"内部把相邻字节两两匹配、计算出现频率

<input type="checkbox" unchecked> BPE-No.3: 合并(Merge) 选取最高频(并列选最高字典序) pair 合并成新 token 重复到 vocab 满

In [1]:
import os
import regex as re 
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from collections import Counter
from cs336_pretokenization_example import find_chunk_boundaries

In [2]:
train_path = "/home/winbeau/Study/1-transformer/datasets/TinyStories/txt/train_with_eot.txt"
assert os.path.exists(train_path), "Not found train_with_eot.txt"

In [3]:
num_processes = min(12, cpu_count())
print(f"Using {num_processes} processes")

Using 12 processes


In [4]:
# regex 正则化 减弱标点、其他符号对文本的影响
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
SPECIAL = "<|endoftext|>" # 结束标志特殊正则

In [11]:
"""不推荐使用这个函数 split 会把原始字符串复制一份"""
def pretokenize_chunk(chunk: str): # 如果不对结尾特殊正则 会出现 .<| 与 |>
    tokens = []
    parts = chunk.split(SPECIAL) # 这将导致内存直接翻倍
    for i, part in enumerate(parts): 
        for match in re.finditer(PAT, part): # 普通文本一般正则
            tokens.append(match.group())
        if i < len(parts) - 1: # 最后一段特殊正则 SPECIAL = "<|endoftext|>" 
            tokens.append(SPECIAL)
    return tokens

`file.seek()` 把文件指针跳到当前块的**起点**

`file.read()` 从**起点**读取当前块的长度

`Counter()` 自动计数器

`group()` 显示分词结果

In [12]:
def process_chunk_(start_end): # 进程处理函数 对 <|endoftext|> 匹配失效
    start, end = start_end 
    counter = Counter()

    with open(train_path, "rb") as f: # 二进制读取 精确跳跃位置
        f.seek(start)
        chunk = f.read(end - start).decode("utf-8", errors="ignore")

    for match in re.finditer(PAT, chunk): # 正则分词
        tokens = match.group()
        counter[tokens.encode("utf-8")] += 1 # 统计为 Unicode bytes

    return counter 

def process_chunk(start_end):
    start, end = start_end
    counter = Counter()

    with open(train_path, "rb") as f:
        f.seek(start)
        chunk = f.read(end - start).decode("utf-8", errors="ignore")

    idx = 0
    while True: # 手动查找 <|endoftext|> 位置
        pos = chunk.find(SPECIAL, idx)
        if pos == -1: # 没找到 对剩下部分用正则分词
            part = chunk[idx:]
            for m in re.finditer(PAT, part):
                tok = m.group()
                counter[tok.encode("utf-8")] += 1
            break
        # 对特殊 token 前面的部分做分词
        part = chunk[idx:pos]
        for m in re.finditer(PAT, part):
            tok = m.group()
            counter[tok.encode("utf-8")] += 1
        # 单独计一次特殊 token
        counter[SPECIAL.encode("utf-8")] += 1
        idx = pos + len(SPECIAL)

    return counter

In [34]:
with open(train_path, "rb") as f: 
    boundaries = find_chunk_boundaries(f, num_processes, b"<|endoftext|>")

chunk_pairs = list(zip(boundaries[:-1], boundaries[1:])) # 0 1 -> 1 2 
print(f"Found {len(chunk_pairs)} chunks")

Found 12 chunks


In [35]:
with Pool(num_processes) as p: 
    counters = list(tqdm(
        p.imap(process_chunk, chunk_pairs), # 若不想加过程可视化模块直接 p.imap 即可
        total=len(chunk_pairs), 
        desc="Pre-tokenization chunks", 
        ncols=80
    ))

total_counts = sum(counters, Counter())
print(f"Total unique tokens: {len(total_counts)}")

Pre-tokenization chunks: 100%|██████████████████| 12/12 [00:57<00:00,  4.79s/it]


Total unique tokens: 72600


In [36]:
print("Top 10 most common tokens:") # 可以看到有很多前导空格————' '频率极高，使用前导空格优化效率
for token, freq in total_counts.most_common(10): 
    try: 
        print(f"{token.decode('utf-8', errors='ignore')!r} : {freq} ")
    except Exception: 
        print(f"{token} : {freq}")
print(f"<|endoftext|> freq: {total_counts.get(b'<|endoftext|>', 0)}")

Top 10 most common tokens:
'.' : 34641640 
' and' : 17810182 
',' : 16629494 
' the' : 16600232 
' to' : 12630263 
' a' : 11358991 
' was' : 9435962 
' ' : 7521334 
' it' : 5172631 
' "' : 5002578 
<|endoftext|> freq: 2119489


## 3-BPE | 统计字节对频率(Pair counting)
<input type="checkbox" checked> BPE-No.1: 预分词(Pre-tokenization) 把原始文本分成初步"词形片段"并计数(正则化去掉标点、符号)

<input type="checkbox" unchecked> BPE-No.2: 统计字节对频率(Pair counting) 在"词形片段"内部把相邻字节两两匹配、计算出现频率

<input type="checkbox" unchecked> BPE-No.3: 合并(Merge) 选取最高频(并列选最高字典序) pair 合并成新 token 重复到 vocab 满

现有 $n$ 个变长字符串 $str_i$ ($len(str_i) \leq 20,~i \in [0, \, n)$)，总长度 $\sum_{i = 0}^n len(str_i) \leq 150 \times 2 \times 10^6 = 3 \times 10^8$，

$vocab$ 为 $token ~ id$ 对 $Unicode ~ byte(s)$ 的映射集合（也就是 $token$ 集合），初始化为 $[0, 256)$ 对应其十六进制数所代表的 $Unicode ~ byte$ (可以理解为 $ASCII ~ plus$) 

**持续执行如下操作：**

对每个字符串 ***所有最长 $token$*** 进行两两结合统计频率：

<aside>

初始状态每个字符是一个 $token$

`word` → `wo` : 1  `or` : 1  `rd` : 1

> 什么是***最长 $token$*** ？如 `newest` ，`est` 已分配 $token ~ id$，我们仅对 `ne` `ew` `west` 处理
> 
</aside>

将频率最高的一个 $pair$ （若并列则取最高字典序）分配新的 $token ~ id$ 

比如上述 `word` 则新分配 `279` → `wo`

**截止状态：**

最终整个序列已经分配了 $token ~ id$（几乎不可能） 或 $token ~ id$ 的数目 = $vocab\_size$

$token ~ id \leq vocab\_size = 10^4$

---

#### 输入

二维字符串数组

#### 输出

$vocab$ : 从 $token ~ id$ 到 $bytes$ 的映射

$merges$ : 产生的 合并 $pair$（按创建顺序排列）

In [5]:
from collections import Counter
from typing import Dict, List, Tuple 

vocab_size = 10000
special_tokens = ["<|endoftext|>"]

#### train_bpe 需要统计全局 pair，但是读取整个文件内存会爆炸，采取策略：
- 全局视野，分块统计
- 只存频率，不存数据
- 一次统计，多次更新

In [6]:
import os, regex
from typing import List, Tuple, Dict
from collections import Counter
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

def process_chunk_freq(args):
    """单个进程：读取一块 -> 正则分词 -> 统计 pair 频率"""
    input_path, start, end, special_tokens = args
    local_counter = Counter()
    with open(input_path, "rb") as f:
        f.seek(start)
        text = f.read(end - start).decode("utf-8", errors="ignore")

    # 按正则模式分词
    for token in regex.findall(PAT, text):
        token = token.strip()
        if not token or token in special_tokens:
            continue
        seq = list(token.encode("utf-8"))
        local_counter.update(zip(seq, seq[1:]))

    return local_counter


def train_bpe_freq(
    input_path: str,
    vocab_size: int,
    special_tokens: List[str],
) -> Tuple[Dict[int, bytes], List[Tuple[bytes, bytes]]]:

    from cs336_pretokenization_example import find_chunk_boundaries  # 你自己写的函数

    num_proc = min(2, cpu_count())
    print(f"Using {num_proc} processes...")

    # === Step 1️⃣: 按 <|endoftext|> 切块 ===
    with open(input_path, "rb") as f:
        boundaries = find_chunk_boundaries(f, num_proc, b"<|endoftext|>")
    chunk_pairs = list(zip(boundaries[:-1], boundaries[1:]))

    # === Step 2️⃣: 并行统计 Counter ===
    with Pool(num_proc) as pool:
        counters = list(tqdm(
            pool.imap(process_chunk_freq,
                      [(input_path, s, e, special_tokens) for s, e in chunk_pairs]),
            total=len(chunk_pairs),
            desc="Pre-tokenization + Counting",
            ncols=80
        ))

    # === Step 3️⃣: 汇总所有 pair 频率 ===
    total_pairs = Counter()
    for c in counters:
        total_pairs.update(c)

    # === Step 4️⃣: 初始化词表 ===
    vocab: Dict[int, bytes] = {i: bytes([i]) for i in range(256)}
    merges: List[Tuple[bytes, bytes]] = []
    next_id = 256
    for tok in special_tokens:
        vocab[next_id] = tok.encode("utf-8")
        next_id += 1

    # === Step 5️⃣: BPE 训练循环 ===
    pbar = tqdm(total=vocab_size - next_id, desc="Training BPE", ncols=80)
    while next_id < vocab_size and total_pairs:
        best_pair, freq = total_pairs.most_common(1)[0]
        if freq < 2:
            break

        # 注册新 token
        new_bytes = vocab[best_pair[0]] + vocab[best_pair[1]]
        vocab[next_id] = new_bytes
        merges.append((vocab[best_pair[0]], vocab[best_pair[1]]))
        next_id += 1

        # 简单版：下一轮重新统计所有 pair（保证正确）
        # 对 10k vocab + TinyStories 可接受
        total_pairs = Counter()
        with Pool(num_proc) as pool:
            counters = list(pool.imap(
                process_chunk_freq,
                [(input_path, s, e, special_tokens) for s, e in chunk_pairs],
            ))
        for c in counters:
            total_pairs.update(c)

        pbar.update(1)

    pbar.close()
    return vocab, merges

In [ ]:
train_path = "/home/winbeau/Study/1-transformer/datasets/TinyStories/txt/train_with_eot.txt"
special_tokens = ["<|endoftext|>"]

vocab, merges = train_bpe_freq(train_path, vocab_size=10000, special_tokens=special_tokens)

Using 2 processes...


Pre-tokenization + Counting:   0%|                        | 0/2 [00:00<?, ?it/s]

In [32]:
list("ac".encode("utf-8"))

[97, 99]